In [31]:
import pyiem.cscap_utils as util
import gdata.spreadsheets.data
import pandas as pd
import datetime

In [59]:
spr_client = util.get_spreadsheet_client(util.get_config())

inspreadsheet = util.Spreadsheet(spr_client, '1tCg0mjZ6LDvS4DVsxDu9NPnvjVo8qPrjnc8_S_kdYlw')
outspreadsheet = util.Spreadsheet(spr_client, '1E7TE4-KmMbcLqKtntB3BsZut__1aF3p3DqFSfwH37rc')

rows = []
for plotid in inspreadsheet.worksheets:
    lf = inspreadsheet.worksheets[plotid].get_list_feed()
    for entry in lf.entry:
        rows.append(entry.to_dict())

df = pd.DataFrame(rows)

In [60]:
df = df[df['date'] != 'Date'].copy()

In [61]:
df['date'] = df['date'].apply(lambda x: datetime.datetime.strptime(x.strip(), '%m/%d/%Y').date())

In [38]:
lf = outspreadsheet.worksheets['2010'].get_list_feed()
for entry in lf.entry:
    print entry.to_dict()

{'sewat8tiletotalpconcentration': '(mg N L-1)', 'swwat20tilenitrate-nloss': '(kg ha-1)', 'newat2tilenitrate-nconcentration': '(mg N L-1)', 'newat20tilenitrate-nloss': '(kg ha-1)', 'newat8tiletotalpconcentration': '(mg N L-1)', 'nwwat8tiletotalpconcentration': '(mg N L-1)', 'nwwat20tilenitrate-nloss': '(kg ha-1)', 'sewat20tilenitrate-nloss': '(kg ha-1)', 'swwat8tiletotalpconcentration': '(mg N L-1)', 'newat9tilereactivepconcentration': '(mg N L-1)', 'sewat9tilereactivepconcentration': '(mg N L-1)', 'swwat2tilenitrate-nconcentration': '(mg N L-1)', 'nwwat2tilenitrate-nconcentration': '(mg N L-1)', 'sewat2tilenitrate-nconcentration': '(mg N L-1)', 'date': 'MM/DD/YYYY', 'nwwat9tilereactivepconcentration': '(mg N L-1)', 'swwat9tilereactivepconcentration': '(mg N L-1)'}


In [62]:
def doo(val):
    if pd.isnull(val):
        return 'did not collect'
    return str(val)

for year in range(2014, 2016):
    frames = {}
    for plotid in ['NW', 'SW', 'SE', 'NE']:
        df2 = df[(df['plotid'] == plotid)&(df['date'] < datetime.date(year +1, 1, 1))&
                (df['date'] >= datetime.date(year, 1, 1))].copy()
        frames[plotid] = df2.set_index('date')
        lv = None
        for i in frames[plotid].index.values:
            if i == lv:
                print i, plotid
            lv = i
    
    df3 = pd.DataFrame(dict(nw_wat2=frames['NW']['wat2'],
                            nw_wat8=frames['NW']['wat8'],
                            nw_wat9=frames['NW']['wat9'],
                            sw_wat2=frames['SW']['wat2'],
                            sw_wat8=frames['SW']['wat8'],
                            sw_wat9=frames['SW']['wat9'],
                            se_wat2=frames['SE']['wat2'],
                            se_wat8=frames['SE']['wat8'],
                            se_wat9=frames['SE']['wat9'],
                            ne_wat2=frames['NE']['wat2'],
                            ne_wat8=frames['NE']['wat8'],
                            ne_wat9=frames['NE']['wat9']))
    for i, row in df3.iterrows():
        entry = gdata.spreadsheets.data.ListEntry()
        entry.set_value('date', i.strftime('%Y-%m-%d'))
        entry.set_value('nwwat2tilenitrate-nconcentration', doo(row['nw_wat2']))
        entry.set_value('nwwat8tiletotalpconcentration', doo(row['nw_wat8']))
        entry.set_value('nwwat9tilereactivepconcentration', doo(row['nw_wat9']))
        entry.set_value('swwat2tilenitrate-nconcentration', doo(row['sw_wat2']))
        entry.set_value('swwat8tiletotalpconcentration', doo(row['sw_wat8']))
        entry.set_value('swwat9tilereactivepconcentration', doo(row['sw_wat9']))
        entry.set_value('sewat2tilenitrate-nconcentration', doo(row['se_wat2']))
        entry.set_value('sewat8tiletotalpconcentration', doo(row['se_wat8']))
        entry.set_value('sewat9tilereactivepconcentration', doo(row['se_wat9']))
        entry.set_value('newat2tilenitrate-nconcentration', doo(row['ne_wat2']))
        entry.set_value('newat8tiletotalpconcentration', doo(row['ne_wat8']))
        entry.set_value('newat9tilereactivepconcentration', doo(row['ne_wat9']))
        spr_client.add_list_entry(entry, outspreadsheet.id,
                                  outspreadsheet.worksheets[str(year)].id)